In [3]:
!pip install --upgrade pillow

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/pip/_vendor/pkg_resources/__init__.py", line 3108, in _dep_map
    return self.__dep_map
  File "/usr/local/lib/python3.10/dist-packages/pip/_vendor/pkg_resources/__init__.py", line 2901, in __getattr__
    raise AttributeError(attr)
AttributeError: _DistInfoDistribution__dep_map

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/base_command.py", line 169, in exc_logging_wrapper
    status = run_func(*args)
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/req_command.py", line 242, in wrapper
    return func(self, options, args)
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/commands/install.py", line 441, in run
    conflicts = self._determine_conflicts(to_install)
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/commands/install.py", line 

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
!git init
!git config --global user.email "rendikarendi96@gmail.com"
!git config --global user.name "Rendika7"

hint: Using 'master' as the name for the initial branch. This default branch name
hint: is subject to change. To configure the initial branch name to use in all
hint: of your new repositories, which will suppress this warning, call:
hint: 
hint: 	git config --global init.defaultBranch <name>
hint: 
hint: Names commonly chosen instead of 'master' are 'main', 'trunk' and
hint: 'development'. The just-created branch can be renamed via this command:
hint: 
hint: 	git branch -m <name>
Initialized empty Git repository in /content/.git/


In [7]:
!ls "/content/drive/MyDrive/Colab Notebooks/Stuntrack"

'Boundingbox Anak.ipynb'	   DeepLearning_Yohanes.ipynb
'Bounding Box Anak YOLOv5.ipynb'  'Hasil Model'
'Data Final'			  'Image Stunting Prediction.ipynb'
'Data Test'			  'Introduction to Computer Vision.ipynb'
'Data Train'			   The_Hello_World_of_Neural_Network_with_TensorFlow.ipynb
'Data Validation'		   yolov3.weights


In [ ]:
import os
from PIL import Image
import cv2
import torch
import tensorflow as tf
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import os, os.path
from PIL import Image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import time
import numpy as np
from google.colab import files
import keras.utils as image
import random
import shutil
from keras.preprocessing import image
from google.colab.patches import cv2_imshow

# Detection part


# import pretrained Model Yolov5S

In [ ]:
model = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True)
device = torch.device(
    'cuda') if torch.cuda.is_available() else torch.device('cpu')
model.to(device).eval()

# Detect People Function

In [ ]:
def detect_people(image):
    img = Image.fromarray(image)
    img = img.convert('RGB')
    # img = img.resize((640, 640))
    results = model(img)
    detections = results.xyxy[0]
    # Filter only people (class index 0)
    detections = detections[detections[:, 5] == 0]
    return detections

# Assign Dataset_Root and get each image file name

In [ ]:
DATASET_ROOT = "/content/drive/MyDrive/Colab Notebooks/Stuntrack/Data Final/Stunting"

image_path = os.listdir(DATASET_ROOT)

jpg_files = [file for file in image_path if file.endswith(".jpg")]
print(jpg_files)
len(jpg_files)

# Manual Rotation Testing

In [ ]:
# Load the image
image_path = "/content/drive/MyDrive/Colab Notebooks/Stuntrack/Data Final/Stunting/TB_Fatar.jpg"
image = cv2.imread(image_path)

rotated_image = cv2.rotate(image, cv2.ROTATE_180)

# Detect people
detections = detect_people(rotated_image)
print(detections)

# Extract bounding box coordinates and confidence
xmin, ymin, xmax, ymax, confidence, _ = detections[0][:6]
print("Confidence:", confidence)

# Draw bounding box on the image
cv2.rectangle(rotated_image, (int(xmin), int(ymin)), (int(xmax), int(ymax)), (0, 0, 255), 5)

# Display the image
cv2_imshow(rotated_image)

# Auto Detect People in Every Rotation for Single Image

In [ ]:
# Load the original image
image_path = "/content/drive/MyDrive/Colab Notebooks/Stuntrack/Data Final/Stunting/TB_Fatar.jpg"
original_image = cv2.imread(image_path)

# List to store results for each rotation
results_per_rotation = []

# Define the rotation angles
rotation_angles = [0, 90, 180, 270]

# Iterate through each rotation angle
for angle in rotation_angles:
    if angle == 90:
        # Rotate the image
        rotated_image = cv2.rotate(original_image, cv2.ROTATE_90_CLOCKWISE)
    elif angle == 180:
        # Rotate the image
        rotated_image = cv2.rotate(original_image, cv2.ROTATE_180)
    elif angle == 270:
        # Rotate the image
        rotated_image = cv2.rotate(original_image, cv2.ROTATE_90_COUNTERCLOCKWISE)
    else:
        # Rotate the image for other angles
        rotated_image = cv2.rotate(original_image, angle)

    # Detect people in the rotated image
    detections = detect_people(rotated_image)

    # Check if detections is not None and not empty
    if detections is not None and len(detections) > 0:
        # Iterate through all detected bounding boxes
        for detection in detections:
            # Extract bounding box coordinates and confidence
            xmin, ymin, xmax, ymax, confidence, _ = detection[:6]

            # Store the results for this rotation
            results_per_rotation.append({
                'angle': angle,
                'confidence': confidence,
                'roi': (int(xmin), int(ymin), int(xmax), int(ymax))
            })

In [ ]:
results_per_rotation

In [ ]:
# Find the rotation with the highest confidence
best_rotation_result = max(results_per_rotation, key=lambda x: x['confidence'])

# Get the best ROI and confidence
best_roi = best_rotation_result['roi']
best_confidence = best_rotation_result['confidence']

# Draw bounding box on the original image using the best ROI
cv2.rectangle(original_image, (best_roi[0], best_roi[1]), (best_roi[2], best_roi[3]), (0, 0, 255), 5)

# Display the original image with the best ROI
cv2_imshow(original_image)


Dalam tensor `[3.64248e+02, 4.40524e+02, 2.39838e+03, 4.44644e+03, 3.08422e-01, 0.00000e+00]`, nilai-nilai tersebut dapat diartikan sebagai berikut:

- x (koordinat x atas kiri bounding box): 3.64248e+02
- y (koordinat y atas kiri bounding box): 4.40524e+02
- lebar bounding box: 2.39838e+03
- tinggi bounding box: 4.44644e+03
- confidence (skor kepercayaan deteksi): 3.08422e-01
- kelas (label deteksi): 0.00000e+00

# Let's Play with not detection Images Apply Auto Detect Auto Rotation to every image we have

In [ ]:
# List to store results for each rotation
all_results = []

# Define the rotation angles
rotation_angles = [0, 90, 180, 270]

# Iterate through each image in the folder
for filename in ['TB_Fatar.jpg', 'TB_KenzoA.jpg', 'TB_MAlFatih.jpg', 'TB_MeccaShabiraShanum.jpg']: # list foto yang tidak nemu di detect people
    if filename.endswith('.jpg'):
        # Load the original image
        image_path = os.path.join(DATASET_ROOT, filename)
        print(f"File {filename} tidak memiliki deteksi bounding box. Lokasi: {image_path}")

        original_image = cv2.imread(image_path)

        # List to store results for each rotation of this image
        results_per_rotation = []

        # Iterate through each rotation angle
        for angle in rotation_angles:
            if angle == 90:
                # Rotate the image
                rotated_image = cv2.rotate(original_image, cv2.ROTATE_90_CLOCKWISE)
            elif angle == 180:
                # Rotate the image
                rotated_image = cv2.rotate(original_image, cv2.ROTATE_180)
            elif angle == 270:
                # Rotate the image
                rotated_image = cv2.rotate(original_image, cv2.ROTATE_90_COUNTERCLOCKWISE)
            else:
                # Rotate the image for other angles
                rotated_image = cv2.rotate(original_image, angle)

            # Detect people in the rotated image
            detections = detect_people(rotated_image)

            # Check if detections is not None and not empty
            if detections is not None and len(detections) > 0:
                # Iterate through all detected bounding boxes
                for detection in detections:
                    # Extract bounding box coordinates and confidence
                    xmin, ymin, xmax, ymax, confidence, _ = detection[:6]

                    # Store the results for this rotation
                    results_per_rotation.append({
                        'angle': angle,
                        'confidence': confidence,
                        'roi': (int(xmin), int(ymin), int(xmax), int(ymax)),
                        'filename': filename  # Add the filename for reference
                    })

        # Append results for this image to the overall results
        all_results.extend(results_per_rotation)
print("\n")
print(f'Hasil BoundingBox In Every Rotation:')
for hasil in all_results:
  print(hasil)

# Main Code

In [ ]:
counter = 0

images_with_no_detections = []

for jpg_file in jpg_files:
    # Construct the full path to the image file
    image_path = os.path.join(DATASET_ROOT, jpg_file)

    # Read the image using cv2
    image = cv2.imread(image_path) # In Matrix Array Form

    # Perform your object detection here (I assume you have a `detect_people` function)
    detections = detect_people(image) # berupa tensor yang berisi informasi tentang orang yang terdeteksi. Berdasarkan output yang diberikan, tampaknya mencakup koordinat bounding box (x, y, lebar, tinggi), skor kepercayaan (0.308), dan mungkin informasi tambahan.

    # If there are detections, take the first one
    if len(detections) > 0:
        detection = detections[0]  # Take the first detection, Karena takut ada double detection

        xmin, ymin, xmax, ymax, confidence, _  = detection[:6] # Akuisisi Komponen dan koordinat hasil deteksi posisi orangnya

        # Draw a rectangle around the detected object on the processed image
        cv2.rectangle(image, (int(xmin), int(ymin)),
                      (int(xmax), int(ymax)), (0, 0, 255), 5) # (0, 0, 255) Warna Hijau BB-nya, dan Width line nya 5 pixel

        output_folder_bb = "/content/drive/MyDrive/Colab Notebooks/Stuntrack/Data Final/BoundingBox Images"
        os.makedirs(output_folder_bb, exist_ok=True)  # Create the folder if it doesn't exist

        # Remove ".jpg" extension from file name
        file_names_without_extension = jpg_file.split('.')[0]

        output_filename_boundingbox = f"{output_folder_bb}/{file_names_without_extension}_{counter}_BoundingBox.jpg" # Generate a unique filename for the processed image

        # Save the processed image with the rectangle
        cv2.imwrite(output_filename_boundingbox, image) # Menyimpan Gambar yang dikasih BoundingBox

        # Crop the detected region of interest (ROI)
        roi = image[int(ymin):int(ymax), int(xmin):int(xmax)]

        output_folder_crop = "/content/drive/MyDrive/Colab Notebooks/Stuntrack/Data Final/Cropped Images"
        os.makedirs(output_folder_crop, exist_ok=True)  # Create the folder if it doesn't exist
        output_filename_cropped = f"{output_folder_crop}/{file_names_without_extension}_{counter}_CroppedROI.jpg" # Generate a unique filename for the processed image

        # Save the cropped ROI as a separate image
        cv2.imwrite(output_filename_cropped, roi)

        # Increment the counter for the next image
        counter += 1
    else:
        images_with_no_detections.append(jpg_file)

        # List to store results for each rotation
        all_results = []

        # Define the rotation angles
        rotation_angles = [0, 90, 180, 270]

        # Iterate through each image in the folder
        angka = 0
        for filename in images_with_no_detections:
          print(f"File {filename} Telah Diolah Lebih lanjut pada Iterasi ke-{angka}. Lokasi: {os.path.join(DATASET_ROOT, filename)}")
          if filename.endswith('.jpg'):
              # Load the original image
              image_path = os.path.join(DATASET_ROOT, filename)

              original_image = cv2.imread(image_path)

              # List to store results for each rotation of this image
              results_per_rotation = []

              # Iterate through each rotation angle
              for angle in rotation_angles:
                  if angle == 90:
                      # Rotate the image
                      rotated_image = cv2.rotate(original_image, cv2.ROTATE_90_CLOCKWISE)
                  elif angle == 180:
                      # Rotate the image
                      rotated_image = cv2.rotate(original_image, cv2.ROTATE_180)
                  elif angle == 270:
                      # Rotate the image
                      rotated_image = cv2.rotate(original_image, cv2.ROTATE_90_COUNTERCLOCKWISE)
                  else:
                      # Rotate the image for other angles
                      rotated_image = cv2.rotate(original_image, angle)

                  # Detect people in the rotated image
                  detections = detect_people(rotated_image)

                  # Check if detections is not None and not empty
                  if detections is not None and len(detections) > 0:
                      # Iterate through all detected bounding boxes
                      for detection in detections:
                          # Extract bounding box coordinates and confidence
                          xmin, ymin, xmax, ymax, confidence, _ = detection[:6]

                          # Store the results for this rotation
                          results_per_rotation.append({
                                'angle': angle,
                                'confidence': confidence,
                                'roi': (int(xmin), int(ymin), int(xmax), int(ymax)),
                                'filename': filename  # Add the filename for reference
                            })

              # Append results for this image to the overall results
              all_results.extend(results_per_rotation)
          angka += 1

print('\n')
print("File Perlu Preprocessing agar memunculkan Boundingbox:")
print(images_with_no_detections)
print('---'*5)
print(f'Hasil BoundingBox In Every Rotation:')
for result in all_results:
  print(result)

from collections import defaultdict

# Dictionary to store the highest confidence for each filename
highest_confidence_dict = defaultdict(float)

# Iterate through all results
for result in all_results:
  filename = result['filename']
  confidence = result['confidence']

  # Update highest confidence for this filename
  highest_confidence_dict[filename] = max(highest_confidence_dict[filename], confidence)

# List to store the final selected results
selected_results = []

# Iterate through all results again and select the ones with highest confidence
for result in all_results:
  filename = result['filename']
  confidence = result['confidence']

  # Check if this result has the highest confidence for its filename
  if confidence == highest_confidence_dict[filename]:
        selected_results.append(result)

# Print the final selected results
for result in selected_results:
  print(result)

In [ ]:
from collections import defaultdict

# Dictionary to store the highest confidence for each filename
highest_confidence_dict = defaultdict(float)

# Iterate through all results
for result in all_results:
    filename = result['filename']
    confidence = result['confidence']

    # Update highest confidence for this filename
    highest_confidence_dict[filename] = max(highest_confidence_dict[filename], confidence)

# List to store the final selected results
selected_results = []

# Iterate through all results again and select the ones with highest confidence
for result in all_results:
    filename = result['filename']
    confidence = result['confidence']

    # Check if this result has the highest confidence for its filename
    if confidence == highest_confidence_dict[filename]:
        selected_results.append(result)

# Print the final selected results
for result in selected_results:
    print(result)

In [ ]:
# Folder to save bounding boxes and cropped images
output_folder_crop = "/content/drive/MyDrive/Colab Notebooks/Stuntrack/Data Final/Cropped Images"
os.makedirs(output_folder_crop, exist_ok=True)
output_folder_bb = "/content/drive/MyDrive/Colab Notebooks/Stuntrack/Data Final/BoundingBox Images"
os.makedirs(output_folder_bb, exist_ok=True)

# Iterate through all selected results
for result in selected_results:
    filename = result['filename']
    angle = result['angle']
    roi = result['roi']

    # Extract bounding box coordinates
    xmin, ymin, xmax, ymax = roi

    # Draw bounding box on the rotated image
    cv2.rectangle(rotated_image, (xmin, ymin), (xmax, ymax), (0, 0, 255), 5)

    # Crop the region of interest
    cropped_image = rotated_image[ymin:ymax, xmin:xmax]

    # Remove ".jpg" extension from file name
    file_names_without_extension = filename.split('.')[0]

    # Save the bounding box image
    output_path_bbox = os.path.join(output_folder_bb, f"{file_names_without_extension}_{counter}_BoundingBox_rotate:{angle}.jpg")
    cv2.imwrite(output_path_bbox, rotated_image)

    # Save the cropped image
    output_path_cropped = os.path.join(output_folder_crop, f"{file_names_without_extension}_{counter}_CroppedROI_rotate:{angle}.jpg")
    cv2.imwrite(output_path_cropped, cropped_image)